# FINM 36700 HW2 solutions

## 1. Conceptual issues for LTCM

### 1.1 Describe LTCM’s investment strategy in less than 100 words.
Many reasonable things to put here. Should identify their reliance on relative value and convergence trades. Should also mention their opportunistic trading as they spot dislocations due to institutional demands. (For example, liquidity needs.)

### 1.2 What are LTCM’s biggest advantages over its competitors?
Several advantages.
- Eﬃcient ﬁnancing. LTCM got very favorable terms on all ﬁnancing—sometimes even zero haircut! Typically had small, if any, outlay.
- Fund size. Have market power even in the large market of institutional wholesale.
- Liquidity. LTCM has in place many mechanisms to ensure liquidity.
- Long-term horizon. In ﬁnancing and assessing trades, LTCM takes a relatively long-term view.
- Hedged. LTCM avoids taking too much default risk or explicit directional bets.

Then again, LTCM went bust in Aug. 1998, so maybe these advantages were not as strong as it seemed!

### 1.3 The case discusses speciﬁc ways in which LTCM manages each of these risks. Brieﬂy discuss them.
The case discusses steps LTCM took to manage four types of funding risks.
• Collateral haircuts. For most trades, LTCM obtains 100% ﬁnancing on a fully collateralized basis. Furthermore, LTCM stress tests the haircuts across its asset classes.
• Repo. LTCM goes against the norm by entering into relatively long-maturity repo. While much of it is overnight, LTCM uses contracts that typically have maturity of 6-12 months. Furthermore, LTCM manages their aggregate repo maturity.
• Equity redemption. The ﬁrm is highly levered, so equity funding risk is especially important. LTCM restricts redemptions of equity year by year. The restriction is particularly strong in that unredeemed money is re-locked. They also spread the redemption windows across the year to ensure there is never a possibility of immediate withdrawal of a large portion of equity.
• For debt funding, LTCM negotiated a revolving loan that has no Material Adverse Change clause. Thus, the availability of debt funding is not so highly correlated with fund performance.

### 1.4 LTCM is largely in the business of selling liquidity and volatility. Describe how LTCM accounts for liquidity risk in their quantitative measurements.
LTCM attempts to account for liquidity risk quantitatively by adjusting security correlations. For short-term horizons, LTCM assumes positive correlation between all trade categories. Even if their net exposure to a strategy ﬂips sides, they still assume positive correlation to the new net position. Given the eﬀorts of LTCM to hedge out obvious market risks, there are many strategies which would seem to have zero correlation. However, LTCM feels that liquidity concerns can cause the eﬀective trading to be positively correlated.

### 1.5 Is leverage risk currently a concern for LTCM?
It would seem that leverage is not particularly dangerous at the moment. The fund’s volatility is relatively low, its VaR is relatively low, nor is it particularly high relative to the rest of the industry. Moreover, the ﬁrm actively manages its funding risk which theoretically means it should be able to handle the natural risks of high leverage. At the time of the case, the ﬁrm is trying to determine whether to further increase leverage. Subsequently, at the end of 1997 the fund returned about a third of its 7.5 billion equity capital to investors. Of course, less than a year later, the fund blew up, but from the time of the case it’s hard to see the leverage risk.


## 2 LTCM Risk

In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
# if not installed, use pip install statsmodels or conda install statsmodels (if you download jupyter from anaconda distribution) 

In [2]:
# read three tabs together
df0 = pd.read_excel('hedge_data.xls', sheet_name = 0, index_col=0)
df1 = pd.read_excel('hedge_data.xls', sheet_name = 1, index_col=0)
df2 = pd.read_excel('hedge_data.xls', sheet_name = 2, index_col=0)
display([df0.index[0],df0.index[-1]])
display([df1.index[0],df1.index[-1]])
display([df2.index[0],df2.index[-1]])

[Timestamp('1994-04-30 00:00:00'), Timestamp('1998-06-30 00:00:00')]

[Timestamp('1994-04-30 00:00:00'), Timestamp('2012-12-31 00:00:00')]

[Timestamp('1994-04-30 00:00:00'), Timestamp('2012-12-31 00:00:00')]

### 2.1 Summary stats

In [14]:
mu = df0[['gross','net']].mean() * 12
vol = df0[['gross','net']].std() * np.sqrt(12)
SR = mu/vol
pd.DataFrame({'mean':mu, 'vol':vol, 'SR':SR})

,mean,vol,SR
gross,0.257153,0.137987,1.863603
net,0.168918,0.113187,1.492373


### 2.2/2.3 Regression of net returns and market returns, and regression metrics

In [18]:
y = df0['net']
X = sm.add_constant(df2['Market Equity Index'].loc[df0.index])  # include intercept in regression, trim the data to the shorter one
res = sm.OLS(y,X).fit()  # drop all the missing data
alpha, beta = res.params
rsquared = res.rsquared
TR = df0['net'].mean()/beta
IR = alpha/res.resid.std()
stats = [alpha, beta, rsquared, TR, IR]
pd.DataFrame(stats, columns = ['net'], index=['alpha', 'beta', 'rsquared', 'TR', 'IR'])

,net
alpha,0.013431
beta,0.042028
rsquared,0.001651
TR,0.334931
IR,0.411390


From the $R^2$ we can tell that there's no much exposure to the market return.

### 2.4 Tail risk

In [27]:
df0['net'].quantile(0.05), df0['net'].nsmallest(5)[-1]  # the 5th quantile and the 5th smallest, just FYI.

(-0.0266, -0.018099999999999998)

In [28]:
(df0['net'].nsmallest(4)).mean()  # mean of 4 worst return

-0.056575

In [29]:
df0['net'].skew()

-0.8573139307755562

In [30]:
df0['net'].kurtosis()

2.9878671881397474

## 3. Other Hedge Fund Indexes

In [44]:
summary_stats = pd.DataFrame()
summary_stats['mean'] = df1.mean()*12
summary_stats['vol'] = df1.std()*12**0.5
summary_stats['skewness'] = df1.skew()
summary_stats['kurtosis'] = df1.kurtosis()
summary_stats['5th percentile'] = df1.quantile(0.05)
summary_stats

,mean,vol,skewness,kurtosis,5th percentile
Total Index,0.060352,0.073219,-0.257508,2.775748,-0.02662
Convertible Arbitrage,0.045781,0.068567,-2.658021,16.445037,-0.02196
Dedicated Short Bias,-0.065563,0.168179,0.690978,1.465212,-0.07568
Emerging Markets,0.051445,0.145699,-0.886015,5.612071,-0.07074
Equity Market Neutral,0.024741,0.101881,-12.075638,167.544417,-0.01298
Event Driven,0.060629,0.062192,-2.313280,11.683685,-0.02644
Event Driven Distressed,0.070747,0.064908,-2.289098,12.053496,-0.02422
Event Driven Multi-Strategy,0.055888,0.067533,-1.789141,8.072271,-0.02698
Event Driven Risk Arbitrage,0.033653,0.040749,-1.109561,5.256679,-0.01528
Fixed Income Arbitrage,0.027131,0.056769,-4.448850,31.550355,-0.01366


In [47]:
regressand = df1.columns
X = sm.add_constant(df2['Market Equity Index'])
stats = []
for col in regressand:
    y = df1[col]
    res = sm.OLS(y,X,missing='drop').fit()
    alpha, beta = res.params
    rsquared = res.rsquared
    TR = y.mean()/beta
    IR = alpha/res.resid.std()
    stats.append((alpha,beta,rsquared,TR,IR))
pd.DataFrame(stats, index = regressand, columns=['alpha', 'beta', 'rsquared', 'TR', 'IR'])

,alpha,beta,rsquared,TR,IR
Total Index,0.003487,0.283865,0.380517,0.017717,0.209630
Convertible Arbitrage,0.002909,0.166726,0.149684,0.022883,0.159404
Dedicated Short Bias,-0.000757,-0.866493,0.672015,0.006305,-0.027218
Emerging Markets,0.001398,0.531952,0.337464,0.008059,0.040822
Equity Market Neutral,0.001033,0.189443,0.087532,0.010883,0.036760
Event Driven,0.003637,0.260557,0.444351,0.019391,0.271775
Event Driven Distressed,0.004474,0.261788,0.411812,0.022520,0.311304
Event Driven Multi-Strategy,0.003227,0.263297,0.384818,0.017689,0.211050
Event Driven Risk Arbitrage,0.002018,0.144703,0.319237,0.019381,0.207963
Fixed Income Arbitrage,0.001607,0.120289,0.113664,0.018796,0.104190
